# Constructing and Running ANNs for RF Mutual Information Investigation
## Thomas Possidente

### Imports and Value Initialization

In [2]:
# ANN Building Imports
import keras
from keras import backend as K
from keras import optimizers
from keras.engine.topology import Layer
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from keras.callbacks import LambdaCallback


# Standard Imports
import pandas as pd
import numpy as np

# Value Inits - Specify as needed
num_inputs_per_batch = int(1000)  # number of dummy images in set
size = int(16)          # Dimension of each dummy image should be size*size
RF_size = int(2)        # Dimensions of the RF to be analyzed should be RF_size*RF_size

# Value Inits - Leave these alone
num_of_RFs = int((size*size) / (RF_size*RF_size))
inputs_by_RF = np.empty([(num_inputs_per_batch * size), size])


### Loading Inputs

In [3]:
inputs = pd.read_csv('test.csv')
inputs = inputs.drop('X1', axis = 0) # Taking out col names
inputs = inputs.apply(pd.to_numeric)  # converting to floats

inputs = inputs.values # convert to np ndarray
inputs = inputs.reshape(1000,16,16) # reshape to desired dims (1000 examples, 16*16)

ValueError: cannot reshape array of size 1280000 into shape (1000,16,16)

### Preprocessing Inputs

In [3]:
# Loop through num_inputs and extract each RF input, flatten, and store in ndarray

origin_x = 0
origin_y = 0
count = 0

for n in range(0, num_of_RFs): # For each RF field in dummy image
    for i in range(0, num_inputs_per_batch): # For each dummy image in set
        single_RF_input = inputs[i, origin_y:(origin_y + RF_size), origin_x:(origin_x + RF_size)] # Extract 1 RF 
        single_RF_input_flat = single_RF_input.reshape((RF_size*RF_size)) 
        inputs_by_RF[count,] = single_RF_input_flat 
        count += 1
    if(origin_x == (size - RF_size)):  # Changes RF field over image
        origin_x = 0
        origin_y += RF_size
    else:
        origin_x += RF_size
        
inputs_by_RF = inputs_by_RF.reshape(num_of_RFs, num_inputs_per_batch, (RF_size*RF_size)) 
# ^Reshaping to (RF Location, Image number, flattened RF) - this makes it easier to select inputs for each separate ANN 

### Building Network

In [49]:
np.set_printoptions(threshold=np.nan)

sess = tf.Session()


class Hebbian(Layer):
    
    
    def __init__(self, output_dim, lmbda=1.0, eta=0.005, winners = 1, connectivity='random', connectivity_prob=0.25, **kwargs):
        '''
        Constructor for the Hebbian learning layer.

        args:
            output_dim - The shape of the output / activations computed by the layer.
            lambda - A floating-point valued parameter governing the strength of the Hebbian learning activation.
            eta - A floating-point valued parameter governing the Hebbian learning rate.
            winners - number of output nodes that should "win"/become activated in winner-take-all activation
            connectivity - A string which determines the way in which the neurons in this layer are connected to
                the neurons in the previous layer.
        '''
        self.output_dim = output_dim
        self.lmbda = lmbda
        self.eta = eta
        self.connectivity = connectivity
        self.connectivity_prob = connectivity_prob

        super(Hebbian, self).__init__(**kwargs)
        
    
    
    def random_conn_init(self, shape, dtype=None):
        A = np.random.normal(0, 1, shape)
        A[self.B] = 0
        return tf.constant(A, dtype=tf.float32)


    def zero_init(self, shape, dtype=None):
        return np.zeros(shape)


    def build(self, input_shape):
        # create weight variable for this layer according to user-specified initialization
        if self.connectivity == 'random':
            self.B = np.random.random(input_shape[0]) < self.connectivity_prob
        elif self.connectivity == 'zero':
            self.B = np.zeros(self.output_dim)
            
        if self.connectivity == 'all':
            self.kernel = self.add_weight(name='kernel', shape=(np.prod(input_shape[1:]), \
                        np.prod(self.output_dim)), initializer='uniform', trainable=False)
        elif self.connectivity == 'random':
            self.kernel = self.add_weight(name='kernel', shape=(np.prod(input_shape[1:]), \
                        np.prod(self.output_dim)), initializer=self.random_conn_init, trainable=False)
        elif self.connectivity == 'zero':
            self.kernel = self.add_weight(name='kernel', shape=(np.prod(input_shape[1:]), \
                        np.prod(self.output_dim)), initializer=self.zero_init, trainable=False)
        else:
            raise NotImplementedError


        # call superclass "build" function
        super(Hebbian, self).build(input_shape)


    def call(self, x):  # x is the input to the network
        batch_size = tf.shape(x)[0]
        x_shape = tf.shape(x)

        # reshape to (batch_size, product of other dimensions) shape
        x = tf.reshape(x, (batch_size, tf.reduce_prod(x_shape[1:])))

        # compute activations using Hebbian-like update rule
        pre_activations = self.lmbda * tf.matmul(x, self.kernel)

        # Should implement winner-takes-all on activations here
        
        numpy_activations = np.zeros((num_inputs_per_batch, self.output_dim))
        
        for n in range(0, num_inputs_per_batch):
            numpy_activation_slice = pre_activations[[n]].eval(session = sess)
            indices_max = np.argsort(-numpy_activation_slice)[:winners]
            numpy_activation_slice = numpy_activation_slice * 0
            numpy_activation_slice[indices_max] = 1
            numpy_activations[n] = numpy_activation_slice
            
        activations = tf.convert_to_tensor(numpy_activations)
                    
        # do it all with tensors - use softmax function with parameter to make everything 0 exept for one value
            #ind = tf.equal(h, tf.reduce_max(h, axis=2, keep_dims=True))
            #return tf.multiply(h, tf.cast(ind, h.dtype))
            
        #    for x in range(0, tf.shape(activations[[n]])):
                
        
        
        
        #activations = x + self.lmbda * tf.matmul(self.kernel, x)  # why "x +", should this be removed? 
                                                                  # Matrix Multiplication will give us the activations
                                                                  # so why add the input to the activation? This shouldn't
                                                                  # even work bc the dims of x are different than
                                                                  # the dims of tf.matmul(kernal, x)

        # compute outer product of activations matrix with itself
            # outer_product = tf.matmul(tf.expand_dims(x, 1), tf.expand_dims(x, 0))  # No idea why you would compute outer prod 
                                                                               # of inputs w/ itself. This method of weight 
                                                                               # updating seems wrong for our purposes
                    
         # update the weight matrix of this layer
            # self.kernel = self.kernel + tf.multiply(self.eta, tf.reduce_mean(outer_product, axis=2)) # Still why outer prod?
            
            
            
            
                    
        # Loop through each input and update weights via hebbian update
        for i in range(0, num_inputs_per_batch):  
            input_set_1 = x[[i]] # select one input
            input_set = tf.tile(input_set_1, [tf.shape(self.kernel)[1]]) # repeat input
            input_set = tf.reshape(input_set, [tf.shape(self.kernel)[1], tf.shape(self.kernel)[0]]) # reshape to transpose of kernal shape
            input_set = tf.transpose(input_set) # transpose
            self.kernel = self.kernel
            #+ (self.eta * (input_set - self.kernel) * activations[[i]]) # weight update based on Hebb's Rule
            # will likely have to build this update into a custom optimization function
       
        #self.kernel = tf.multiply(self.kernel, self.B) # zeroing node connections that started at 0
        return K.reshape(activations, (batch_size, self.output_dim))
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)
        
        

In [48]:
### Scrap Page ###
pre_activations = tf.random_uniform(shape = [1000,4], minval = 0, maxval = 1)
numpy_activations = np.zeros((1000,4))



with tf.Session() as sess:
    numpy_activations_slice = np.array(pre_activations[[0]])
    indices_max = np.argsort(-numpy_activations_slice)[:3]
    #numpy_activations_slice = numpy_activations_slice * 0
    #numpy_activations_slice[indices_max] = 1
    #numpy_activations[0] = numpy_activations_slice
    #activations = tf.convert_to_tensor(numpy_activations)
    print(indices_max)
    sess.close()

[0]


In [50]:
model = Sequential()
model.add(Hebbian(input_shape = (1, (RF_size*RF_size)), output_dim = 4, eta = 0.05, winners = 1))
model.summary()

InvalidArgumentError: You must feed a value for placeholder tensor 'hebbian_4_input' with dtype float and shape [?,1,16]
	 [[Node: hebbian_4_input = Placeholder[dtype=DT_FLOAT, shape=[?,1,16], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'hebbian_4_input', defined at:
  File "C:\Users\Tom\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Tom\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Tom\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Tom\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Tom\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 478, in start
    self.io_loop.start()
  File "C:\Users\Tom\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  

In [15]:
def dummy_loss(y_true, y_pred): return y_pred             

class dummy_opt(keras.optimizers.Optimizer): 
    def __init__(self): return(None)
    def get_updates(self, loss, params): return(np.array(1))
    def get_configs(self): return(0)
    
dummyOpt = dummy_opt()

model.compile(optimizer= dummyOpt,loss = dummy_loss)

In [66]:

print_weights = LambdaCallback(on_epoch_end=lambda epoch, logs: print(model.layers[0].get_weights()))
    

history = model.fit(x = inputs_by_RF[0].reshape(1000, 1, 16), y = np.array([0]*1000), epochs = 10, verbose = 2, callbacks = [print_weights])
history

Epoch 1/10
 - 5s - loss: 2.2507
[array([[ 0.0127599 ,  0.37172297,  0.6517402 , -0.19584408],
       [ 0.48289487,  1.4250616 ,  0.829136  , -0.9990712 ],
       [ 0.75429446, -0.43105736, -0.49123842, -0.916771  ],
       [-0.5936044 , -0.28183797,  0.9929673 , -0.37372187],
       [-2.5864205 ,  0.6306306 , -0.5808915 , -0.19506797],
       [-0.7374694 ,  1.4341315 ,  1.0903759 , -1.2998435 ],
       [-0.19822815,  0.9991844 , -0.36946616, -0.78892595],
       [-0.4907518 ,  0.8508978 ,  0.0213118 ,  0.23771958],
       [ 0.5244177 ,  0.57069975, -0.09198719,  1.6109691 ],
       [ 1.1668501 ,  0.19878256, -0.054135  ,  0.7204712 ],
       [ 0.22814113,  0.95615643, -0.38133034, -1.178827  ],
       [-1.1019708 ,  0.6887924 , -0.10015344,  0.9302938 ],
       [ 1.0542237 , -0.41386527,  1.3975052 ,  1.872338  ],
       [ 1.0615239 ,  0.05719159,  0.16392286,  0.08765532],
       [ 0.93025565,  1.4915359 ,  0.16463295, -1.0579904 ],
       [ 1.4901214 ,  0.18857561, -1.3429745 ,  0.79

[array([[ 0.0127599 ,  0.37172297,  0.6517402 , -0.19584408],
       [ 0.48289487,  1.4250616 ,  0.829136  , -0.9990712 ],
       [ 0.75429446, -0.43105736, -0.49123842, -0.916771  ],
       [-0.5936044 , -0.28183797,  0.9929673 , -0.37372187],
       [-2.5864205 ,  0.6306306 , -0.5808915 , -0.19506797],
       [-0.7374694 ,  1.4341315 ,  1.0903759 , -1.2998435 ],
       [-0.19822815,  0.9991844 , -0.36946616, -0.78892595],
       [-0.4907518 ,  0.8508978 ,  0.0213118 ,  0.23771958],
       [ 0.5244177 ,  0.57069975, -0.09198719,  1.6109691 ],
       [ 1.1668501 ,  0.19878256, -0.054135  ,  0.7204712 ],
       [ 0.22814113,  0.95615643, -0.38133034, -1.178827  ],
       [-1.1019708 ,  0.6887924 , -0.10015344,  0.9302938 ],
       [ 1.0542237 , -0.41386527,  1.3975052 ,  1.872338  ],
       [ 1.0615239 ,  0.05719159,  0.16392286,  0.08765532],
       [ 0.93025565,  1.4915359 ,  0.16463295, -1.0579904 ],
       [ 1.4901214 ,  0.18857561, -1.3429745 ,  0.7978848 ]],
      dtype=float32)]


In [92]:
model.predict(inputs_by_RF[0][0].reshape(1,1,16))

array([[ 1.2465706 ,  1.6297497 , -0.04945281, -3.4030924 ]],
      dtype=float32)

## Notes/TODO

* Implement winner-take-all - how to change values in activation tensor - trying to convert to numpy then back isn't working and can't figure out how to do it without numpy bc of cannot assign new value to indices. 
* Weights not changing (example below) - is it even possible to change weights inside the call method? That's what the original layer did... but I don't know if it actually works. If not, how do I change the weights with a custom optimization function?

* Figure out how to connect many small NNs to output layer
* Metric for measuring learning (mutual information)

* Possibilities: 1) Continue pushing with unsupervised. 2) Switch to supervised - but what would be the ground truth for first layers of network? How would they backprop from the ground truth of the whole image to the small sections given to each network? 
* 1D convolutional NN - stride equal to size of RF. Filter #? - start with number of diff patterns to make it easy, can change from there.


* Note for later - use alphabet dataset (), use samples instead of whole thing for measuring max MI of filter size to speed up learning. 

In [55]:
from keras import backend as K
from keras.layers import Layer

class MyLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        self.kernel = self.kernel + 100
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [56]:
model = Sequential()
model.add(MyLayer(input_shape = (1, 1), output_dim = 1))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_layer_10 (MyLayer)        (None, 1)                 1         
Total params: 1
Trainable params: 1
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.compile(optimizer = dummyOpt, loss = dummy_loss)

In [58]:
print_weights = LambdaCallback(on_epoch_end=lambda epoch, logs: print(model.layers[0].get_weights()))
model.fit(x = np.ones(10).reshape(10,1,1), y=np.zeros(10), epochs = 5, verbose = 2, callbacks = [print_weights])

Epoch 1/5
 - 0s - loss: 100.0390
[array([[0.03902756]], dtype=float32)]
Epoch 2/5
 - 0s - loss: 100.0390
[array([[0.03902756]], dtype=float32)]
Epoch 3/5
 - 0s - loss: 100.0390
[array([[0.03902756]], dtype=float32)]
Epoch 4/5
 - 0s - loss: 100.0390
[array([[0.03902756]], dtype=float32)]
Epoch 5/5
 - 0s - loss: 100.0390
[array([[0.03902756]], dtype=float32)]


### TODOS
* Run again with 2*2, 3*3, 4*4, 5*5, 6*6, 7*7, 8*8 and 10%, 20%, 30%, 40%, 50% noise. Save outputs and Loss curve
* Controls - Work on creating image sets of specific submatrix pattern size (3*3) that have varying levels of MI
* Find Image dataset that's binarized (or binarize existing one) and do MI calculations to find max MI. To do this, sample RF sized chunks of an image until stable average MI value is reached